## Techniche - Topic Modelling

In [37]:
import pandas as pd
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.corpora import mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel, atmodel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import sys

from test_model import tokenize_docs, clean_docs, lower_words, remove_stopwords, get_topics  # TODO (Lee) convert_bytes

from smart_open import smart_open

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

import pyspark
import pyspark.sql.functions as F

In [59]:
%load_ext autoreload

# pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 50
pd.set_option('display.max_rows', 10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
np.random.seed(3)

In [61]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

### Import Data

#### Import data from PatentsView API

In [62]:
# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("/Users/lee/Documents/techniche/techniche/data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()
# pat_fields = ['appcit_app_number', 'appcit_category', 'appcit_date', 'appcit_kind', 'appcit_sequence',
#      'app_country', 'app_date', 'app_number', 'app_type', 'assignee_city', 'assignee_country',
#      'assignee_county', 'assignee_county_fips', 'assignee_first_name', 'assignee_first_seen_date',
#      'assignee_id', 'assignee_last_name', 'assignee_last_seen_date', 'assignee_lastknown_city',
#      'assignee_lastknown_country', 'assignee_lastknown_latitude', 'assignee_lastknown_location_id',
#      'assignee_lastknown_longitude', 'assignee_lastknown_state', 'assignee_latitude', 
#      'assignee_location_id', 'assignee_longitude', 'assignee_organization', 'assignee_sequence',
#      'assignee_state', 'assignee_state_fips', 'assignee_total_num_inventors', 
#      'assignee_total_num_patents', 'assignee_type', 'cited_patent_category', 'cited_patent_date',
#      'cited_patent_kind', 'cited_patent_number', 'cited_patent_sequence', 'cited_patent_title',
#      'citedby_patent_category', 'citedby_patent_date', 'citedby_patent_kind',
#      'citedby_patent_number', 'citedby_patent_title', 'cpc_category', 'cpc_first_seen_date',
#      'cpc_group_id', 'cpc_group_title', 'cpc_last_seen_date', 'cpc_section_id', 'cpc_sequence',
#      'cpc_subgroup_id', 'cpc_subgroup_title', 'cpc_subsection_id', 'cpc_subsection_title',
#      'cpc_total_num_assignees', 'cpc_total_num_inventors', 'cpc_total_num_patents',
#      'detail_desc_length', 'examiner_first_name', 'examiner_id', 'examiner_last_name',
#      'examiner_role', 'examiner_group', 'forprior_country', 'forprior_date', 'forprior_docnumber',
#      'forprior_kind', 'forprior_sequence', 'govint_contract_award_number', 'govint_org_id',
#      'govint_org_level_one', 'govint_org_level_two', 'govint_org_level_three', 'govint_org_name',
#      'govint_raw_statement', 'inventor_city', 'inventor_country', 'inventor_county',
#      'inventor_county_fips', 'inventor_first_name', 'inventor_first_seen_date', 'inventor_id',
#      'inventor_last_name', 'inventor_last_seen_date', 'inventor_lastknown_city',
#      'inventor_lastknown_country', 'inventor_lastknown_latitude', 'inventor_lastknown_location_id',
#      'inventor_lastknown_longitude', 'inventor_lastknown_state', 'inventor_latitude',
#      'inventor_location_id', 'inventor_longitude', 'inventor_sequence', 'inventor_state',
#      'inventor_state_fips', 'inventor_total_num_patents', 'ipc_action_date', 'ipc_class',
#      'ipc_classification_data_source', 'ipc_classification_value', 'ipc_first_seen_date',
#      'ipc_last_seen_date', 'ipc_main_group', 'ipc_section', 'ipc_sequence', 'ipc_subclass',
#      'ipc_subgroup', 'ipc_symbol_position', 'ipc_total_num_assignees', 'ipc_total_num_inventors',
#      'ipc_version_indicator', 'lawyer_first_name', 'lawyer_first_seen_date', 'lawyer_id',
#      'lawyer_last_name', 'lawyer_last_seen_date', 'lawyer_organization', 'lawyer_sequence',
#      'lawyer_total_num_assignees', 'lawyer_total_num_inventors', 'lawyer_total_num_patents',
#      'nber_category_id', 'nber_category_title', 'nber_first_seen_date', 'nber_last_seen_date',
#      'nber_subcategory_id', 'nber_subcategory_title', 'nber_total_num_assignees',
#      'nber_total_num_inventors', 'nber_total_num_patents', 'patent_abstract',
#      'patent_average_processing_time', 'patent_date', 'patent_firstnamed_assignee_city',
#      'patent_firstnamed_assignee_country', 'patent_firstnamed_assignee_id', 
#      'patent_firstnamed_assignee_latitude', 'patent_firstnamed_assignee_location_id',
#      'patent_firstnamed_assignee_longitude', 'patent_firstnamed_assignee_state',
#      'patent_firstnamed_inventor_city', 'patent_firstnamed_inventor_country',
#      'patent_firstnamed_inventor_id', 'patent_firstnamed_inventor_latitude',
#      'patent_firstnamed_inventor_location_id', 'patent_firstnamed_inventor_longitude',
#      'patent_firstnamed_inventor_state', 'patent_kind', 'patent_num_cited_by_us_patents',
#      'patent_num_claims', 'patent_num_combined_citations', 'patent_num_foreign_citations',
#      'patent_num_us_application_citations', 'patent_num_us_patent_citations', 'patent_number',
#      'patent_processing_time', 'patent_title', 'patent_type', 'patent_year', 'pct_102_date',
#      'pct_371_date', 'pct_date', 'pct_docnumber', 'pct_doctype', 'pct_kind',
#      'rawinventor_first_name', 'rawinventor_last_name', 'uspc_first_seen_date',
#      'uspc_last_seen_date', 'uspc_mainclass_id', 'uspc_mainclass_title', 'uspc_sequence',
#      'uspc_subclass_id', 'uspc_subclass_title', 'uspc_total_num_assignees', 
#      'uspc_total_num_inventors', 'uspc_total_num_patents', 'wipo_field_id','wipo_field_title',
#      'wipo_sector_title','wipo_sequence']

#### Import initial dataset

In [63]:
# build query - initial small dataset
query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},{"_text_phrase":{"patent_abstract":"natural language"}}]}
fields=pat_fields
options={"per_page":100}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
response = requests.get(endpoint_url, params=params)
status = response.status_code
print("status:", status)
results = response.json()
count = results.get("count")
total_pats = results.get("total_patent_count")
print("patents on current page:",count,';', "total patents:",total_pats)

status: 200
patents on current page: 100 ; total patents: 2482


#### Structure data

In [64]:
# extract metadata from response
print("status code:", response.status_code,';', "reason:", response.reason)
total_patent_count = results["total_patent_count"]
patents_per_page = results['count']
print("total_patent_count:",total_patent_count,';', "patents_per_page:", patents_per_page)

# extract data from response
data_resp = results['patents']
# data_resp[0]

raw_df = pd.DataFrame(data_resp)
raw_df.head(3)

status code: 200 ; reason: OK
total_patent_count: 2482 ; patents_per_page: 100


,IPCs,application_citations,applications,assignees,cited_patents,citedby_patents,cpcs,detail_desc_length,examiners,foreign_priority,gov_interests,inventors,lawyers,nbers,patent_abstract,patent_average_processing_time,patent_date,patent_firstnamed_assignee_city,patent_firstnamed_assignee_country,patent_firstnamed_assignee_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_state,patent_firstnamed_inventor_city,patent_firstnamed_inventor_country,patent_firstnamed_inventor_id,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_location_id,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_state,patent_kind,patent_num_cited_by_us_patents,patent_num_claims,patent_num_combined_citations,patent_num_foreign_citations,patent_num_us_application_citations,patent_num_us_patent_citations,patent_number,patent_processing_time,patent_title,patent_type,patent_year,pct_data,rawinventors,uspcs,wipos
0,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020077823', 'ap...","[{'app_country': 'US', 'app_date': '2013-07-26...","[{'assignee_city': 'Burlington', 'assignee_cou...",[{'cited_patent_category': 'cited by examiner'...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",11570,"[{'examiner_first_name': 'Michael N', 'examine...","[{'forprior_country': None, 'forprior_date': N...","[{'govint_contract_award_number': None, 'govin...","[{'inventor_city': 'Newton', 'inventor_country...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...",Designing a natural language understanding (NL...,None,2019-03-12,Burlington,US,org_ID497r4tFbCIaMBjGAST,42.5047,42.5047|-71.1961,-71.1961,MA,Newton,US,7788103-1,42.3369,42.3369|-71.2097,-71.2097,MA,B2,0,19,31,0,26,5,10229106,2055,Initializing a workspace for building a natura...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Jeffrey N.', 'raw...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
1,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020138265', 'ap...","[{'app_country': 'US', 'app_date': '2017-09-11...","[{'assignee_city': 'Mountain View', 'assignee_...",[{'cited_patent_category': 'cited by applicant...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",28118,"[{'examiner_first_name': 'Shreyans A', 'examin...","[{'forprior_country': None, 'forprior_date': N...","[{'govint_contract_award_number': None, 'govin...","[{'inventor_city': 'Adliswil', 'inventor_count...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...","Methods, systems, and apparatus, including com...",None,2019-03-12,Mountain View,US,org_p6ofWD2xFNSnyYkj6wpA,37.3861,37.3861|-122.0828,-122.083,CA,Adliswil,CH,8352247-1,47.3119,47.3119|8.5287,8.5287,None,B1,0,20,15,0,7,8,10229109,547,Allowing spelling of arbitrary words,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Evgeny A.', 'rawi...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
2,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2001/20010029455', 'ap...","[{'app_country': 'US', 'app_date': '2016-09-28...","[{'assignee_city': 'Seattle', 'assignee_countr...",[{'cited_patent_category': 'cited by applicant...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",119654,"[{'examiner_first_name': 'Jialong', 'examiner_...","[{'forprior_country': None, 'forprior_date': N...","[{'govint_contract_award_number': None, 'govin...","[{'inventor_city': 'Seattle', 'inventor_countr...","[{'lawyer_first_name': None, 'lawyer_first_see...","[

#### Subset dataframe

In [65]:
# subset dataframe - comment/uncomment to include fields
df = raw_df[['patent_number', 
         'patent_date', 
         'patent_title',
         'patent_abstract', 
         'patent_firstnamed_assignee_id',
         'patent_firstnamed_assignee_location_id',
         'patent_firstnamed_assignee_latitude',
         'patent_firstnamed_assignee_longitude',
         'patent_firstnamed_assignee_city',
         'patent_firstnamed_assignee_state',
         'patent_firstnamed_assignee_country', 
         'patent_firstnamed_inventor_id',
         'patent_firstnamed_inventor_location_id',
         'patent_firstnamed_inventor_latitude',
         'patent_firstnamed_inventor_longitude',
         'patent_firstnamed_inventor_city',
         'patent_firstnamed_inventor_state',
         'patent_firstnamed_inventor_country',
         'patent_year', 
         'patent_type', 
         'patent_kind',
         'inventors'
            ]]
df.head(3)

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_city,patent_firstnamed_assignee_state,patent_firstnamed_assignee_country,patent_firstnamed_inventor_id,patent_firstnamed_inventor_location_id,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_city,patent_firstnamed_inventor_state,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors
0,10229106,2019-03-12,Initializing a workspace for building a natura...,Designing a natural language understanding (NL...,org_ID497r4tFbCIaMBjGAST,42.5047|-71.1961,42.5047,-71.1961,Burlington,MA,US,7788103-1,42.3369|-71.2097,42.3369,-71.2097,Newton,MA,US,2019,utility,B2,"[{'inventor_city': 'Newton', 'inventor_country..."
1,10229109,2019-03-12,Allowing spelling of arbitrary words,"Methods, systems, and apparatus, including com...",org_p6ofWD2xFNSnyYkj6wpA,37.3861|-122.0828,37.3861,-122.083,Mountain View,CA,US,8352247-1,47.3119|8.5287,47.3119,8.5287,Adliswil,None,CH,2019,utility,B1,"[{'inventor_city': 'Adliswil', 'inventor_count..."
2,10229113,2019-03-12,Leveraging content dimensions during the trans...,A content management system (CMS) and a transl...,org_Vbc6obpnxWM42d0HjlXY,47.6064|-122.3308,47.6064,-122.331,Seattle,WA,US,9177341-1,47.6064|-122.3308,47.6064,-122.331,Seattle,WA,US,2019,utility,B1,"[{'inventor_city': 'Seattle', 'inventor_countr..."


#### Explore data

In [66]:
# 561 different assignees
len(df.patent_firstnamed_assignee_id.unique())

39

#### Create new column

In [67]:
# create new column that combines the patent title and the patent abstract columns into a single string
df['patent_title_abstract'] = df.patent_title + ' ' + df.patent_abstract
df.patent_title_abstract.head(3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    Initializing a workspace for building a natura...
1    Allowing spelling of arbitrary words Methods, ...
2    Leveraging content dimensions during the trans...
Name: patent_title_abstract, dtype: object

In [68]:
#### TODO (Lee) Partition data into train and test sets

In [69]:
df.sort_values(by=['patent_date'])

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_city,patent_firstnamed_assignee_state,patent_firstnamed_assignee_country,patent_firstnamed_inventor_id,patent_firstnamed_inventor_location_id,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_city,patent_firstnamed_inventor_state,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors,patent_title_abstract
99,10162315,2018-12-25,Process control system using typical and adapt...,"Methods, systems, and non-transitory, computer...",org_Y5Tht7WZIuEYKAJwDAZQ,40.4406|-79.9961,40.4406,-79.9961,Pittsburgh,PA,US,4433971-2,40.6014|-79.76,40.6014,-79.76,Tarentum,PA,US,2018,utility,B2,"[{'inventor_city': 'Tarentum', 'inventor_count...",Process control system using typical and adapt...
78,10168988,2019-01-01,Identifying user preferences and changing sett...,"A method, a computer program product, and a co...",org_q9Bn28RHhpYrQjKvraAH,41.1264|-73.7144,41.1264,-73.7144,Armonk,NY,US,10168988-1,39.9042|116.4074,39.9042,116.407,Beijing,None,CN,2019,utility,B2,"[{'inventor_city': 'Beijing', 'inventor_countr...",Identifying user preferences and changing sett...
79,10169323,2019-01-01,Diagnosing autism spectrum disorder using natu...,Embodiments herein include a natural language ...,org_q9Bn28RHhpYrQjKvraAH,41.1264|-73.7144,41.1264,-73.7144,Armonk,NY,US,8676787-1,44.0692|-92.7556,44.0692,-92.7556,Mantorville,MN,US,2019,utility,B2,"[{'inventor_city': 'Rochester', 'inventor_coun...",Diagnosing autism spectrum disorder using natu...
80,10169326,2019-01-01,Cognitive reminder notification mechanisms for...,A data processing system generates a result of...,org_q9Bn28RHhpYrQjKvraAH,41.1264|-73.7144,41.1264,-73.7144,Armonk,NY,US,7490098-1,35.8233|-78.8258,35.8233,-78.8258,Morrisville,NC,US,2019,utility,B2,"[{'inventor_city': 'Morrisville', 'inventor_co...",Cognitive reminder notification mechanisms for...
81,10169327,2019-01-01,Cognitive reminder notification mechanisms for...,A data processing system generates a result of...,org_q9Bn28RHhpYrQjKvraAH,41.1264|-73.7144,41.1264,-73.7144,Armonk,NY,US,7490098-1,35.8233|-78.8258,35.8233,-78.8258,Morrisville,NC,US,2019,utility,B2,"[{'inventor_city': 'Morrisville', 'inventor_co...",Cognitive reminder notification mechanisms for...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,10229687,2019-03-12,Scalable endpoint-dependent natural language u...,A computer-implemented technique is described ...,org_EilEWQcC6UiqHcSGx9mb,47.6742|-122.1203,47.6742,-122.12,Redmond,WA,US,5712675-3,47.6419|-122.0792,47.6419,-122.079,Sammamish,WA,US,2019,utility,B2,"[{'inventor_city': 'Sammamish', 'inventor_coun...",Scalable endpoint-dependent natural language u...
12,10230677,2019-03-12,Identifying an entity associated with an onlin...,An approach is described for identifying an en...,org_q9Bn28RHhpYrQjKvraAH,41.1264|-73.7144,41.1264,-73.7144,Armonk,NY,US,7792904-1,42.6056|-83.15,42.6056,-83.15,Troy,MI,US,2019,utility,B2,"[{'inventor_city': 'Sewickley', 'inventor_coun...",Identifying an entity associated with an onlin...
13,10230680,2019-03-12,Intelligently splitting text in messages poste...,"A method, system and computer program product ...",org_q9Bn28RHhpYrQjKvraAH,41.1264|-73.7144,41.1264,-73.7144,Armonk,NY,US,8380487-1,42.6611|-70.9972,42.6611,-70.9972,Boxford,MA,US,2019,utility,B2,"[{'inventor_city': 'Shanghai', 'inventor_count...",Intelligently splitting text in messages poste...
7,10229673,2019-03-12,System and method for providing follow-up resp...,"In certain implementations, follow-up response...",org_9D8x1qL3IRASp6GG7Glu,47.6106|-122.1994,47.6106,-122.199,Bellevue,WA,US,7398209-3,47.6106|-122.1994,47.6106,-122.199,Bellevue,WA,US,2019,utility,B2,"[{'inventor_city': 'Bellevue', 'in

In [71]:
text_data = df.patent_title_abstract.tolist()
text_data

['Initializing a workspace for building a natural language understanding system Designing a natural language understanding (NLU) model for an application from scratch can be difficult for non-experts. A system can simplify the design process by providing an interface allowing a designer to input example usage sentences and build an NLU model based on presented matches to those example sentences. In one embodiment, a method for initializing a workspace for building an NLU system includes parsing a sample sentence to select at least one candidate stub grammar from among multiple candidate stub grammars. The method can include presenting, to a user, respective representations of the candidate stub grammars selected by the parsing of the sample sentence. The method can include enabling the user to choose one of the respective representations of the candidate stub grammars. The method can include adding to the workspace a stub grammar corresponding to the representation of the candidate stu

In [86]:
# partition data
len(text_data)
text_train = text_data[:round(len(text_data)*.8)]
text_test = text_data[round(len(text_data)*.8):]
print(len(text_data), len(text_train), len(text_test), len(text_train)+len(text_test))

100 80 20 100


In [87]:
### Pre-process text data

# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

# construct pipeline using Spacy Language object and associated pipeline/components
nlp = spacy.load("en")
pprint(nlp.pipeline)

processed_docs = []   

# process patent documents in pipeline
for doc in nlp.pipe(text_train, n_threads=4, batch_size=100):
   
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    doc = [token for token in doc if token not in stop_words]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)

processed_docs[0][:5]

[token.text for token in doc]

labels = set([w.label_ for w in doc.ents]) 

for label in labels: 
    entities = [cleanup(e.string, lower=False) for e in document.ents if label==e.label_] 
    entities = list(set(entities)) 
    print(label,entities)

pre_processed_docs = []
for doc in nlp.pipe(docs, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    #doc = [token for token in doc if token not in STOPWORDS]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    pre_processed_docs.append(doc)

[('tagger', <spacy.pipeline.pipes.Tagger object at 0x1a351b8710>),
 ('parser', <spacy.pipeline.pipes.DependencyParser object at 0x1a3341c528>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x1a3341c588>)]


/anaconda3/lib/python3.6/runpy.py:193: DeprecationWarning: [W016] The keyword argument `n_threads` on the is now deprecated, as the v2.x models cannot release the global interpreter lock. Future versions may introduce a `n_process` argument for parallel inference via multiprocessing.
  "__main__", mod_spec)


NameError: name 'stop_words' is not defined

#### Tokenize

In [88]:
# tokenize documents

def tokenize_docs(docs):
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(word_tokenize(doc))
    return tokenized_docs

tokenized_docs = tokenize_docs(text_train)

#### Clean punctuation

In [89]:
# clean punctuation
def clean_docs(tokenized_docs):
    clean_docs = []
    for doc in tokenized_docs:
       clean_docs.append([word for word in doc if word.isalpha()])  
    return clean_docs

In [90]:
cleaned_data = clean_docs(tokenized_docs)
cleaned_data[0]

['Initializing',
 'a',
 'workspace',
 'for',
 'building',
 'a',
 'natural',
 'language',
 'understanding',
 'system',
 'Designing',
 'a',
 'natural',
 'language',
 'understanding',
 'NLU',
 'model',
 'for',
 'an',
 'application',
 'from',
 'scratch',
 'can',
 'be',
 'difficult',
 'for',
 'A',
 'system',
 'can',
 'simplify',
 'the',
 'design',
 'process',
 'by',
 'providing',
 'an',
 'interface',
 'allowing',
 'a',
 'designer',
 'to',
 'input',
 'example',
 'usage',
 'sentences',
 'and',
 'build',
 'an',
 'NLU',
 'model',
 'based',
 'on',
 'presented',
 'matches',
 'to',
 'those',
 'example',
 'sentences',
 'In',
 'one',
 'embodiment',
 'a',
 'method',
 'for',
 'initializing',
 'a',
 'workspace',
 'for',
 'building',
 'an',
 'NLU',
 'system',
 'includes',
 'parsing',
 'a',
 'sample',
 'sentence',
 'to',
 'select',
 'at',
 'least',
 'one',
 'candidate',
 'stub',
 'grammar',
 'from',
 'among',
 'multiple',
 'candidate',
 'stub',
 'grammars',
 'The',
 'method',
 'can',
 'include',
 'presen

#### Convert to lowercase

In [91]:
# convert to lowercase
def lower_words(docs):
    lowered_words = []
    for doc in docs:
        lowered_words.append([word.lower() for word in doc])
    return lowered_words

lowered_data = lower_words(cleaned_data)
lowered_data[0]

['initializing',
 'a',
 'workspace',
 'for',
 'building',
 'a',
 'natural',
 'language',
 'understanding',
 'system',
 'designing',
 'a',
 'natural',
 'language',
 'understanding',
 'nlu',
 'model',
 'for',
 'an',
 'application',
 'from',
 'scratch',
 'can',
 'be',
 'difficult',
 'for',
 'a',
 'system',
 'can',
 'simplify',
 'the',
 'design',
 'process',
 'by',
 'providing',
 'an',
 'interface',
 'allowing',
 'a',
 'designer',
 'to',
 'input',
 'example',
 'usage',
 'sentences',
 'and',
 'build',
 'an',
 'nlu',
 'model',
 'based',
 'on',
 'presented',
 'matches',
 'to',
 'those',
 'example',
 'sentences',
 'in',
 'one',
 'embodiment',
 'a',
 'method',
 'for',
 'initializing',
 'a',
 'workspace',
 'for',
 'building',
 'an',
 'nlu',
 'system',
 'includes',
 'parsing',
 'a',
 'sample',
 'sentence',
 'to',
 'select',
 'at',
 'least',
 'one',
 'candidate',
 'stub',
 'grammar',
 'from',
 'among',
 'multiple',
 'candidate',
 'stub',
 'grammars',
 'the',
 'method',
 'can',
 'include',
 'presen

#### Clean stopwords

In [92]:
# clean stopwords

stop_words = stopwords.words('english')

In [93]:
def filter_stopwords(docs):
    filtered_docs = []
    for doc in docs:
       filtered_docs.append([word for word in doc if word not in stop_words])
    return filtered_docs

# remove stopwords
filtered_data = filter_stopwords(lowered_data)
filtered_data
# TODO (Lee) - resolve un-lowered stopwords "A" and "An", 'By', 'The'

[['initializing',
  'workspace',
  'building',
  'natural',
  'language',
  'understanding',
  'system',
  'designing',
  'natural',
  'language',
  'understanding',
  'nlu',
  'model',
  'application',
  'scratch',
  'difficult',
  'system',
  'simplify',
  'design',
  'process',
  'providing',
  'interface',
  'allowing',
  'designer',
  'input',
  'example',
  'usage',
  'sentences',
  'build',
  'nlu',
  'model',
  'based',
  'presented',
  'matches',
  'example',
  'sentences',
  'one',
  'embodiment',
  'method',
  'initializing',
  'workspace',
  'building',
  'nlu',
  'system',
  'includes',
  'parsing',
  'sample',
  'sentence',
  'select',
  'least',
  'one',
  'candidate',
  'stub',
  'grammar',
  'among',
  'multiple',
  'candidate',
  'stub',
  'grammars',
  'method',
  'include',
  'presenting',
  'user',
  'respective',
  'representations',
  'candidate',
  'stub',
  'grammars',
  'selected',
  'parsing',
  'sample',
  'sentence',
  'method',
  'include',
  'enabling',
 

In [94]:
#### Create corpus and dictionary

# using spacy pipeline components
# build dictionary
id_to_word = corpora.Dictionary(processed_docs)

# build corpus
texts = processed_docs

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(doc) for doc in processed_docs]

 # build dictionary
id_to_word = corpora.Dictionary(filtered_data)

# build corpus
texts = filtered_data

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(text) for text in texts]

# view formatted corpus (term-doc-frequency)
[[(id_to_word[id], freq) for id, freq in text] for text in corpus][:1]

[[('adding', 1),
  ('allowing', 1),
  ('among', 1),
  ('application', 1),
  ('based', 1),
  ('build', 1),
  ('building', 2),
  ('candidate', 5),
  ('choose', 1),
  ('chosen', 1),
  ('corresponding', 1),
  ('design', 1),
  ('designer', 1),
  ('designing', 1),
  ('difficult', 1),
  ('embodiment', 1),
  ('enabling', 1),
  ('example', 2),
  ('grammar', 3),
  ('grammars', 3),
  ('include', 3),
  ('includes', 1),
  ('initializing', 2),
  ('input', 1),
  ('interface', 1),
  ('language', 2),
  ('least', 1),
  ('matches', 1),
  ('method', 4),
  ('model', 2),
  ('multiple', 1),
  ('natural', 2),
  ('nlu', 3),
  ('one', 3),
  ('parsing', 2),
  ('presented', 1),
  ('presenting', 1),
  ('process', 1),
  ('providing', 1),
  ('representation', 1),
  ('representations', 2),
  ('respective', 2),
  ('sample', 2),
  ('scratch', 1),
  ('select', 1),
  ('selected', 1),
  ('sentence', 2),
  ('sentences', 2),
  ('simplify', 1),
  ('stub', 6),
  ('system', 3),
  ('understanding', 2),
  ('usage', 1),
  ('user'

In [95]:
### Model - model #1

# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus,
                     id2word=id_to_word,
                     num_topics=25, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

# print keywords in n topics
pprint(model_lda.print_topics())

# print top 10 keywords that comprise topic with index of 0
pprint(model_lda.print_topic(24))
# the most import keywords, and the respective weight, that form topic 0 are

# print top 10 keywords that comprise topic with index of 1
pprint(model_lda.print_topic(1))

# TODO (Lee) - infer topic from keywords?

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

[(5,
  '0.001*"message" + 0.001*"segments" + 0.001*"messages" + 0.001*"user" + '
  '0.001*"posted" + 0.001*"based" + 0.001*"website" + 0.001*"social" + '
  '0.001*"media" + 0.001*"meaning"'),
 (1,
  '0.029*"knowledge" + 0.029*"database" + 0.029*"queries" + 0.029*"manager" + '
  '0.029*"priority" + 0.029*"scores" + 0.019*"structured" + 0.019*"contexts" + '
  '0.019*"upon" + 0.019*"multiple"'),
 (9,
  '0.033*"search" + 0.024*"entered" + 0.020*"recently" + 0.020*"term" + '
  '0.020*"candidates" + 0.020*"language" + 0.020*"natural" + 0.016*"query" + '
  '0.016*"keyword" + 0.016*"needs"'),
 (15,
  '0.031*"one" + 0.031*"sound" + 0.023*"language" + 0.022*"least" + '
  '0.022*"electronic" + 0.022*"resources" + 0.022*"communication" + '
  '0.018*"natural" + 0.018*"embodiments" + 0.018*"interface"'),
 (20,
  '0.038*"language" + 0.038*"natural" + 0.029*"computing" + 0.029*"user" + '
  '0.029*"device" + 0.029*"intent" + 0.029*"intents" + 0.024*"image" + '
  '0.024*"capture" + 0.024*"general"'),
 (

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

In [ ]:
### Evaluate - model #1

# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus)
perplexity

# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=processed_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=filtered_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

In [100]:
# explore topics
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus, id_to_word)
viz_topics_1
# TODO (Lee) - salient vs relevant terms in pyLDA ?

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.148238 -0.074501       1        1  10.997526
4      0.096287  0.044581       2        1   9.095231
23     0.053027  0.108729       3        1   6.659494
17     0.077369 -0.038001       4        1   5.672835
18     0.133585 -0.021303       5        1   5.177598
...         ...       ...     ...      ...        ...
20    -0.010220  0.155188      21        1   2.590135
7     -0.210108  0.016059      22        1   2.411487
13    -0.090679 -0.060119      23        1   1.023706
1     -0.114449 -0.027378      24        1   0.897262
5     -0.131010 -0.032387      25        1   0.025067

[25 rows x 5 columns], topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
88    Default   56.000000   content   56.000000  30.0000  30.0000
53    Default  102.000000      user  102.000000  29.0000  29.0000
168   Default   29.000000  response   29.000000  28.0000  28.0000
164   Default   56.000000     query   56.000000  27.0000  27.0000
194   Default  126.000000       may  126.000000  26.0000  26.0000
...       ...         ...       ...         ...      ...      ...
153   Topic25    0.001291     using   42.011368  -2.0987  -7.0928
152   Topic25    0.001284      upon    8.242291  -0.4755  -7.0982
30    Topic25    0.001283  multiple    9.063068  -0.5711  -7.0989
366   Topic25    0.001282      text   17.735672  -1.2436  -7.1000
50    Topic25    0.001277    system  101.150482  -2.9881  -7.1035

[1386 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
414      22  0.654839  abstraction
580       9  0.432448       access
580      10  0.216224       access
580      18  0.216224       access
580      23  0.216224       access
...     ...       ...          ...
1118      6  0.800064     wireless
1045     18  0.643817       wishes
786       8  0.611341      without
586       9  0.726256     workflow
54        5  0.894238    workspace

[2033 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 5, 24, 18, 19, 4, 15, 12, 23, 17, 22, 13, 7, 20, 10, 25, 3, 16, 9, 1, 21, 8, 14, 2, 6])

### Model 2-  Mallet model

In [101]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update this path
path_mallet = 'data/mallet-2.0.8/bin/mallet'

In [106]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus_train, num_topics=25, id2word=id_to_word)

NameError: name 'corpus_train' is not defined

In [107]:
# topics
pprint(model_2.show_topics(formatted=False))

NameError: name 'model_2' is not defined

In [108]:
# calculate coherence metric
coherence_model_2 = CoherenceModel(model=model_2, texts=data, dictionary=id_to_word, coherence='c_v')
coherence_model_2 = coherence_model_2.get_coherence()
coherence_model_2

NameError: name 'model_2' is not defined

In [109]:
# TODO (Lee)
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id_to_word, corpus=corpus, texts=data, start=2, limit=40, step=6)

In [ ]:
### Model 3 - Author topic model

# construct inventor-to-doc mapping as df from nested inventors column in json api response
df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
df_inventors.sort_values(by=['patent_date'])
df_inventors.head(3)

df.head(3)

# TODO (Lee) - resolve workaround
df_idx = df
df_idx['idx'] = df.index
df_idx
df_idx_1 = df_idx[['patent_number', 'idx', 'inventors']]
df_idx_2 = df_idx_1.set_index('patent_number')
df_idx_2.pop('inventors')
df_idx_2
df_pat_idx = df_idx_2.T.to_dict('records')
for i in df_pat_idx:
    df_pat_idx = dict(i)
df_pat_idx

df_pat_idx = df_idx_2.T.to_dict('records')
for i in df_pat_idx:
    df_pat_idx = dict(i)
df_pat_idx

df_inv_test = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inv_test.head(3)

df_idx_pat_inv_map = df[['patent_number', 'inventors']]
df_idx_pat_inv_map.head(3)

# TODO (Lee) - find out how to get list of patents_view_field names from API - I did it accidentally but need to replicate

df.patent_title_abstract[0]

df[:3]

df_inventors.set_index('inventor_id').T.to_dict('list')

# for k, v in pat2inv.items():
#     name_dict[new_key] = name_dict.pop(k)
#     time.sleep(4)

# pprint.pprint(name_dict)

# d = {'x':1, 'y':2, 'z':3}
# d1 = {'x':'a', 'y':'b', 'z':'c'}

# dict((d1[key], value) for (key, value) in d.items())
# {'a': 1, 'b': 2, 'c': 3}

patdf2inv = dict((df_pat_idx[key], value) for (key, value) in pat2inv.items())
patdf2inv

pat2inv = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
pat2inv

idx_pat_map = df.patent_number.to_dict()
idx_pat_map = {str(key): value for key, value in idx_pat_map.items()}
idx_pat_map

#### Construct author-topic model

# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus,
                         doc2author=patdf2inv,
                         id2word=id_to_word, 
                         num_topics=25)

# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

# retrieve the topic distribution for an author using use model[name] syntax
# each topic has a probability of being expressed given the particular author, but only the ones above a certain threshold are shown.

model_at['7788103-1']

# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

# calculate per-word bound, which is a measure of the model's predictive performance (reconstruction error?)

build doc2author dictionary

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

from gensim.models import atmodel
doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

gensim.models.atmodel.construct_author2doc(doc2author)
# construct mapping from author IDs to document IDs.

Parameters:	doc2author (dict of (int, list of str)) – Mapping of document id to authors.
Returns:	Mapping of authors to document ids.
Return type:	dict of (str, list of int)

gensim.models.atmodel.construct_doc2author(corpus, author2doc)
construct mapping from document IDs to author IDs

Parameters:	
corpus (iterable of list of (int, float)) – Corpus in BoW format.
author2doc (dict of (str, list of int)) – Mapping of authors to documents.
Returns:	
Document to Author mapping.

Return type:	
dict of (int, list of str)

### Appendix

#### Import full dataset from PatentsView API

In [7]:
# uncomment to use

# def get_patents_by_month(begin_date,end_date, pats_per_page):
#     """ requests patent data from PatentsView API by date range"""
#     endpoint_url = 'http://www.patentsview.org/api/patents/query'
#     page_counter=1
#     data = []
#     results = {}
#     count=1
    
#     for i in range(round(100000/pats_per_page)): # TODO (Lee) - replace with datetime for begin_date to end_date
        
#         if count ==0:
#             print("error/complete")
#             break
            
#         elif count > 0:     
#             # build query
#             query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
#             fields=pat_fields
#             options={"page": page_counter, "per_page":pats_per_page}
#             sort=[{"patent_date":"desc"}]
#             params={'q': json.dumps(query),
#                     'f': json.dumps(fields),
#                     'o': json.dumps(options),
#                     's': json.dumps(sort)
#                         }
    
#             # request and results
#             response = requests.get(endpoint_url, params=params)
#             status = response.status_code
#             print("status:", status,';',"page_counter:",page_counter, ";", "iteration:",i)
#             results = response.json()
#             count = results.get("count")
#             total_pats = results.get("total_patent_count")
#             print("patents on current page:",count,';', "total patents:",total_pats)
#             data.extend(results)
#             page_counter+=1
        
#     return data
#             # TODO (Lee) results =  json.loads(response.content)
#             # TODO (Lee) places.extend(results['results'])
#             # TODO (Lee) time.sleep(2)

CPC fields for block 1 of query:
Y10S-706 OR 
G06N-003 OR 
G06N-005/003:G06N-005/027 OR 
G06N- 007/005:G06N-007/06 OR 
G06N-099/005 OR
G06T2207/20081 OR
G06T2207/20084 OR
G06T-003/4046 OR
G06T-009/002 OR
G06F-017/16 OR
G05B-013/027 OR
G05B- 013/0275 OR
G05B-013/028 OR
G05B-013/0285 OR
G05B-013/029 OR
G05B-013/0295 OR
G05B-2219/33002 OR
G05D-001/0088 OR
G06K-009 OR
G10L-015 OR
G10L-017 OR
G06F-017/27:G06F-017/2795 OR
G06F-017/28:G06F-017/289 OR
G06F-017/30029:G06F- 017/30035 OR
G06F-017/30247:G06F-017/30262 OR 
G06F-017/30401 OR
G06F-017/3043 OR 
G06F-017/30522:G06F-017/3053 OR 
G06F-017/30654 OR 
G06F-017/30663 OR
G06F-017/30666 OR 
G06F-017/30669 OR
G06F-017/30672 OR 
G06F-017/30684 OR
G06F-017/30687 OR 
G06F-017/3069 OR 
G06F-017/30702 OR
G06F-017/30705:G06F- 017/30713 OR
G06F-017/30731:G06F-017/30737 OR
G06F-017/30743:G06F-017/30746 OR 
G06F-017/30784:G06F-017/30814 OR
G06F-019/24 OR G06F-019/707 OR
G01R- 031/2846:G01R-031/2848 OR
G01N-2201/1296 OR
G01N-029/4481 OR
G01N-033/0034 ORG01R-031/3651ORG01S-007/417ORG06N-003/004:G06N-003/008 ORG06F- 011/1476 OR 
G06F-011/2257 OR 
G06F-011/2263 OR 
G06F-015/18 OR
G06F-2207/4824 OR
G06K-007/1482 OR
G06N-007/046 OR
G11B-020/10518 OR
G10H-2250/151 OR
G10H-2250/311 OR
G10K-2210/3024 OR
H01J-2237/30427 OR
H01M-008/04992 OR
H02H-001/0092 OR
H02P-021/0014 OR
H02P-023/0018 OR
H03H-2017/0208 OR
H03H- 2222/04 OR
H04L-2012/5686 OR
H04L-2025/03464 OR
H04L-2025/03554 OR
H04L- 025/0254 OR
H04L-025/03165 OR
H04L-041/16 OR
H04L-045/08 OR
H04N- 021/4662:H04N-021/4666 OR
H04Q-2213/054 
OR H04Q-2213/13343 OR
H04Q-2213/343 OR
H04R-025/507 OR
G08B-029/186 OR
B60G-2600/1876 OR
B60G-2600/1878 OR
B60G-2600/1879 OR
B64G-2001/247 OR
E21B-2041/0028 OR
B23K-031/006 OR
B29C- 2945/76979 OR
B29C-066/965 OR
B25J-009/161 OR
A61B-005/7264:A61B-005/7267 OR
Y10S-128/924 OR
Y10S-128/925 OR
F02D-041/1405 OR
F03D-007/046 OR
F05B- 2270/707 OR
F05B-2270/709 OR
F16H-2061/0081 OR
F16H-2061/0084 OR
B60W-030/06 OR
B60W-030/10:B60W-030/12 OR
B60W-030/14:B60W-030/17 OR
B62D-015/0285 OR
G06T-2207/30248:G06T-2207/30268 OR
G06T-2207/30236 OR G05D-001 OR
A61B- 005/7267 OR
F05D-2270/709 OR
G06T-2207/20084 OR
G10K-2210/3038 OR
G10L-025/30 OR
H04N-021/4666 OR
A63F-013/67 OR
G06F-017/2282

In [ ]:
#### Import data from bulk download

# uncomment to download TSV files containing detailed patent descriptions from PatentsView 
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2016.tsv.zip # 2016 - 3.0 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2017.tsv.zip # 2017 - 2.8 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2018.tsv.zip # 2018 - 1.6 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2019.tsv.zip # 2019 - 0.7 GB zipped

# !unzip files
# unzip detail-desc-text-2016.tsv.zip
# unzip detail-desc-text-2017.tsv.zip
# unzip detail-desc-text-2018.tsv.zip
# unzip detail-desc-text-2019.tsv.zip

# def convert_bytes(num, suffix='B'):
#     """ convert bytes int to int in aggregate units"""
#     for unit in ['','K','M','G','T','P','E','Z']:
#         if abs(num) < 1024.0:
#             return "%3.1f%s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f%s%s" % (num, 'Yi', suffix)

# path = "data/"
# with os.scandir(path) as it:
#     for entry in it:
#         if not entry.name.startswith('.') and entry.is_file():
#             print(entry.name)

# # inspect unzipped file sizes

# convert_bytes(os.path.getsize("data/detail-desc-text-2016.tsv"))
# convert_bytes(os.path.getsize("data/detail-desc-text-2017.tsv"))
# convert_bytes(os.path.getsize("data/detail-desc-text-2018.tsv"))
# convert_bytes(os.path.getsize("data/detail_desc_text_2019.tsv"))

#### Spark workflow

# create SparkSession/SparkContext as entry point to Dataset/DataFrame API

# spark = pyspark.sql.SparkSession.builder.getOrCreate()
# sc = spark.sparkContext
# sc

# doc by doc read of large tsv files

# with open "data/detail-desc-text-2018.tsv" as f_in:
#     with open('data/2018_shard.tsv', 'w') as f_out:
#         for line in f_in:
#             if contains_keywords(line):
#                 f_out.write(line)

# from functools import reduce

# df = reduce(lambda x,y: x.unionAll(y), 
#             [spark.read.format('csv')
#                        .load(f, header="true", inferSchema="true") 
#              for f in files])
# df.show()

# files = ["data/detail-desc-text-2016.tsv", "data/detail-desc-text-2017.tsv", 
#          "data/detail-desc-text-2018.tsv", "data/detail-desc-text-2019.tsv"]

# uncomment to use

# df_2018 = (spark.read
#                .format("csv")
#                .option("delimiter", "\t")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2018.tsv")
#                .write
#                .format("parquet")
#                .save("df_2018.parquet"))

# df_2018 = (spark.read
#                .format("csv")
#                .option("delimiter", "\t")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2018.tsv"))

# dfp_2018 = pd.read_csv("data/detail-desc-text-2018.tsv", sep='\t', header=None)

# dfp_2018.columns = ['patent_number', 'desc_detail', 'len_detail']

# dfp_2018_nl.head(3)

# dfp_2018_nl = dfp_2018[dfp_2018['desc_detail'].str.contains('NLP')]

# df_2018.printSchema()

# schema = StructType([
#             StructField("_c0", IntegerType(), True),
#             StructField("_c1", StringType(), True),
#             StructField("_c2", IntegerType(), True)])

# df_2018 = spark.read.load('data/df_2018.parquet')

# # 160,249 rows in 2018 dataset
# df_2018.count()

# df_2018.rdd.getNumPartitions()

# # partition / batching ?
# df_2018.filter(df_2018._c1.contains("natural language")).count()

# query file directly with SQL

# query = """
# SELECT * FROM parquet.`data/df_2018.parquet` WHERE _c1 LIKE 'natural language' LIMIT 100
# """

# df_2018_nl = spark.sql(query)

# df_2018_nl.head(3)

# df_2018.columns

# df_2018.explain()

# df_2018.describe().show()

# df_2018.dtypes

# df_171819 = df_2017.union(df_2018).union(df_2019)

# df_171819.count()

# df_171819.head(3)

# df_2018.head(3)

# counts = df_2018.agg(F.countDistinct('_c0'))
# counts

# reviews_df.createOrReplaceTempView('reviews')

# output = spark.sql(query)

# show(output, n=1000)

# results = spark.sql(
#   "SELECT * FROM people")
# names = results.map(lambda p: p.name)

# df.rdd.isEmpty()

# df = (spark.read
#             .load("data/*.parquet")
#             .write
#             .format("parquet")
#             .save("df.parquet"))

# df_2019 = (spark.read
#                .format("csv")
#                .option("delimiter", "\t")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2018.tsv")
#                .write
#                .format("parquet")
#                .save("df_2019.parquet"))

# df_2019.head(3)

# type(df_2018)

# df_2016.head(3)

# df_2016 = (spark.read.format("csv")
#                .option("delimiter", "\t")
#                .option("header", "true")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2016.tsv")
#                .write
#                .format("parquet")
#                .save("data/df_2016.parquet"))

# df_2016.head(3)

# df = (spark.read.format("csv")
#            .option("delimiter", ",")
#            .infer
#            .load("data/df.csv"))

# df_2017 = (spark.read.format("csv")
#                .option("delimiter", "\t")
#                .option("header", "true")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2017.tsv")
#                .write
#                .format("parquet")
#                .save("df_2017.parquet"))

# df_2018 = spark.read.parquet("data/df_2018.parquet")

# df_2018.head(2)

# df_2018.persist()

# df_2018.take(2)

# df_2018.toPandas()

In [ ]:
#### Construct bigrams and trigrams

# train bigram phrases model
bigram_model = Phrases(filtered_data, min_count=1, threshold=1)

# train trigram phrases model
trigram_model = Phrases(bigram_model[filtered_data], threshold=100)  

# bigrams
def bigrams(docs):
    """create bigrams"""
    return [bigram_model[doc] for doc in docs]

# initialize bigram and trigram models
bigram_model = gensim.models.phrases.Phraser(bigram_model)
trigram_model = gensim.models.phrases.Phraser(trigram_model)

bigrams(filtered_data)[0]

def trigrams(docs):
    """create trigrams"""
    return [trigram_model[bigram_model[doc]] for doc in docs]

trigrams(filtered_data)[0]

#### Stem and Lemmatize

In [ ]:
def lemmatize_docs(docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """lemmatize documents"""
    lemmatized_docs = []
    for doc in docs: 
        lemmatized_docs.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return lemmatized_docs

In [ ]:
# TODO (Lee)

lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
# for doc in cleaned_data:
#     for token in doc:
#         token.lemma_

# uncomment to use
# download english model with "python -m spacy download en"

# for token in doc:
#     print(token, token.lemma, token.lemma_)

# TODO (Lee) - lemmatize_docs(cleaned_data)